In [17]:
import psycopg2
import pandas as pd
from pandas import DataFrame
import pandas.io.sql as psql

In [140]:
#connect to the db
con = psycopg2.connect(
            host = "localhost",
            database = "chicago_business_intelligence",
            user = "postgres",
            password = "root",
            port = 5433)

In [141]:
cur = con.cursor()

In [65]:
#requirement 3 
#grab HIGH ccvi category data
cur.execute("select community_area_or_zip, community_area_name, ccvi_category from ccvi_data")

rows = cur.fetchall()

df = pd.DataFrame(rows)
df.columns = ['community_area_or_zip', 'community_area_name', 'ccvi_category']
df['ccvi_category'] = df['ccvi_category'].astype(str).astype(float)

#filter out areas that are not HIGH on CCVI scale 
df = df[df['ccvi_category'] > 47.4]

#grab taxi trip pickup and drop off data
cur.execute("select pickup_zip_code, dropoff_zip_code from taxi_trips")

rows_tt = cur.fetchall()

df_tt = pd.DataFrame(rows_tt)
df_tt.columns = ['pickup_zip_code', 'dropoff_zip_code']

#create a flag that checks for trips that pick up or drop off in a area with a HIGH CCVI
df_tt = df_tt.assign(flag_p=df_tt.pickup_zip_code.isin(df.community_area_or_zip).astype(int))
df_tt = df_tt.assign(flag_d=df_tt.dropoff_zip_code.isin(df.community_area_or_zip).astype(int))

In [70]:
total = df_tt['flag_p'].sum() + df_tt['flag_d'].sum()
print("The total number of trips in the dataset that include a pick up or drop of to an area with a HIGH CCVI are", total)

The total number of trips in the dataset that include a pick up or drop of to an area with a HIGH CCVI are 102


In [144]:
#requirement 5
cur.execute("select community_area, community_area_name, unemployment, below_poverty_level from community_area_unemployment")

rows = cur.fetchall()

df_unemployment = pd.DataFrame(rows)
df_unemployment.columns = ['community_area', 'community_area_name', 'unemployment_rate', 'poverty_rate']

#select top 5 unemployment rate areas and top 5 poverty rate areas
df_top5_unemployment = df_unemployment.sort_values(by='unemployment_rate', ascending=False)
df_top5_unemployment = df_top5_unemployment.head()
df_top5_poverty = df_unemployment.sort_values(by='poverty_rate', ascending=False)
df_top5_poverty = df_top5_poverty.head()

#pull relevant data from building permits
cur.execute("select permit_id, community_area, subtotal_unpaid, total_fee, contact_1_type, contact_1_name, contact_1_city, contact_1_state, contact_1_zipcode  from building_permits")

rows = cur.fetchall()

df_bp = pd.DataFrame(rows)
df_bp.columns = ['permit_id', 'community_area', 'subtotal_unpaid', 'total_fee', 'contact_1_type', 'contact_1_name', 'contact_1_city', 'contact_1_state', 'contact_1_zipcode']

#check for open building permits in areas with high poverty rates or high unemployment and flag them
df_bp = df_bp.assign(flag_u=df_bp.community_area.isin(df_top5_unemployment).astype(int))
df_bp = df_bp.assign(flag_p=df_bp.community_area.isin(df_top5_poverty).astype(int))

#create temp dfs to hold flagged data
temp1 = df_bp[(df_bp.flag_u == 1)]
temp2 = df_bp[(df_bp.flag_p == 1)]

#display which open building permits should have fees waived
df_waive_fee = temp1.append(temp2)
print("Here is a list of every open building permit in an area with top 5 poverty or top 5 unemployment:")
df_waive_fee

Here is a list of every open building permit in an area with top 5 poverty or top 5 unemployment:


,permit_id,community_area,subtotal_unpaid,total_fee,contact_1_type,contact_1_name,contact_1_city,contact_1_state,contact_1_zipcode,flag_u,flag_p


In [148]:
print(df_top5_poverty)
print(df_top5_unemployment)
df_bp.community_area.unique()

   community_area community_area_name unemployment_rate poverty_rate
61             62         West Elsdon              13.5          9.8
69             70             Ashburn               8.8          9.5
75             76              O'Hare               4.7          9.5
3               4      Lincoln Square               6.8          9.5
55             56      Garfield Ridge               8.1            9
   community_area community_area_name unemployment_rate poverty_rate
63             64            Clearing               9.6          5.9
54             55           Hegewisch               9.6         12.1
20             21            Avondale               9.3         14.6
76             77           Edgewater                 9         16.6
10             11      Jefferson Park                 9          6.4


array(['61', '1', '17', '10', '29', '76', '65', '15', '25', '3', '43',
       '66', '8', '28', '33', '34', '13', '16', '77', '24', '32', '57',
       '6', '22', '75'], dtype=object)

In [142]:
#requirement 2
#zip code for MDW is 60638 and ORD is 60018
cur.execute("select trip_start_timestamp, pickup_zip_code, dropoff_zip_code from taxi_trips")

rows = cur.fetchall()

df_air = pd.DataFrame(rows)
df_air.columns = ['trip_start_timestamp', 'pickup_zip_code', 'dropoff_zip_code']
#to expand on this you would extract the week number from the timestamp and then compare to how the weekly covid cases were during the following period

#find all trips that went to or from either airport
temp1 = df_air[(df_air.pickup_zip_code == '60638')]
temp2 = df_air[(df_air.pickup_zip_code == '60018')]
temp3 = df_air[(df_air.dropoff_zip_code == '60638')]
temp4 = df_air[(df_air.dropoff_zip_code == '60018')]

df_air_taxi = temp1.append(temp2)
df_air_taxi = df_air_taxi.append(temp3)
df_air_taxi = df_air_taxi.append(temp4)

#get covid detail data
cur.execute("select zip_code, week_number, cases_weekly from covid_data")

rows = cur.fetchall()

df_covid = pd.DataFrame(rows)
df_covid.columns = ['zip_code', 'week_number', 'cases_weekly']

print("The following dataset includes all taxi trips that either went to the airport or came from the airport")
print(df_air_taxi)

print("The following dataset includes the relevant COVID weekly case rate for each zip code")
print(df_covid)
#to further expand you would want to see which area codes had the highest weekly rates of travel to and from the airport and then sort out those zipcodes in the covid data

The following dataset includes all taxi trips that either went to the airport or came from the airport
         trip_start_timestamp pickup_zip_code dropoff_zip_code
10  2022-11-01 00:00:00+00:00           60638            60612
12  2022-11-01 00:00:00+00:00           60638            60610
17  2022-11-01 00:00:00+00:00           60638            60616
18  2022-11-01 00:00:00+00:00           60638            60610
23  2022-11-01 00:00:00+00:00           60638            60614
..                        ...             ...              ...
360 2022-10-31 22:45:00+00:00           60638            60638
424 2019-04-26 16:45:00+00:00           60629            60638
426 2019-03-31 13:00:00+00:00           60611            60638
493 2019-06-09 13:00:00+00:00           60657            60638
793 2019-04-16 19:30:00+00:00           60614            60638

[73 rows x 3 columns]
The following dataset includes the relevant COVID weekly case rate for each zip code
    zip_code week_number cases_we

In [139]:
#close connection
con.close()